## HomeWork к уроку 5 

> **"Первая нейронная сеть на Tensorflow"** 


In [1]:
# %tensorflow_version 2.x
%load_ext tensorboard
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as layers
import tensorflow.keras.callbacks as callbacks
import numpy as np
import matplotlib.pyplot as plt

<b>Настройка GPU</b>

In [2]:
# кастомные настройки
# установка GPU на котором будем работать
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[3], 'GPU')
tf.config.get_visible_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

##### Загрузка данных
<b>Загрузка данных</b>

In [143]:
(__x_train, y_train), (__x_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [144]:
# предобработка 
x_train = (np.reshape(__x_train, (__x_train.shape[0], -1)) / 255. - 0.5) * 2
x_test = (np.reshape(__x_test, (__x_test.shape[0], -1)) / 255. - 0.5) * 2

## Код для создания и обучения модели

**Важно!** Вы можете менять код функции `get_model`, если хотите получить лучшие результаты или просто поэксперементировать.

In [5]:
def get_model(n_classes):
  activation = 'tanh'
  units = 256
  model = keras.Sequential([
    layers.Dense(units, activation=activation),
    layers.Dense(units, activation=activation),
    layers.Dense(n_classes, activation='softmax')])
  return model

In [6]:
model = get_model(10)
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01),
              metrics=['accuracy'],
              loss='sparse_categorical_crossentropy')

In [7]:
early_stopping = callbacks.EarlyStopping(min_delta=0.001, patience=3, 
                                         restore_best_weights=True)

tensorboard = tf.keras.callbacks.TensorBoard(log_dir='logs/'+ 'base_model')

epochs = 100
batch_size = 128
histrory = model.fit(x_train, y_train, 
                      validation_data=(x_test, y_test),
                      epochs=epochs,
                      batch_size=batch_size,
                      callbacks=[early_stopping])

Epoch 1/100
469/469 [==============================] - 2s 5ms/step - loss: 0.7611 - accuracy: 0.7461 - val_loss: 0.5792 - val_accuracy: 0.7970
Epoch 2/100
469/469 [==============================] - 2s 4ms/step - loss: 0.5187 - accuracy: 0.8180 - val_loss: 0.5165 - val_accuracy: 0.8144
Epoch 3/100
469/469 [==============================] - 3s 5ms/step - loss: 0.4704 - accuracy: 0.8334 - val_loss: 0.4839 - val_accuracy: 0.8295
Epoch 4/100
469/469 [==============================] - 2s 5ms/step - loss: 0.4431 - accuracy: 0.8436 - val_loss: 0.4706 - val_accuracy: 0.8308
Epoch 5/100
469/469 [==============================] - 3s 5ms/step - loss: 0.4242 - accuracy: 0.8500 - val_loss: 0.4484 - val_accuracy: 0.8387
Epoch 6/100
469/469 [==============================] - 2s 5ms/step - loss: 0.4096 - accuracy: 0.8558 - val_loss: 0.4434 - val_accuracy: 0.8425
Epoch 7/100
469/469 [==============================] - 2s 5ms/step - loss: 0.3979 - accuracy: 0.8596 - val_loss: 0.4296 - val_accuracy: 0.8470

# Задание 1. Слой для подготовки данных

Создайте слой, который:
* получает на вход батч `(None, 28, 28)` и возвращает батч
`(None, 784)`. 
* Приводит значения интенсивности пикселей из диапазона `[0, 255]` к диапазону `[-1, 1]`.
* *Дополнительно*: сделайте слой сериализуемым.


Шаблон с кодом приведен ниже. Обучите модель с использованием этого слоя, убедитесь, что она обучается корректно.

In [46]:
class DataPreparation(layers.Layer):
  def __init__(self, *args, **kwargs):
    # Самодостаточен, дописывать не надо.
    # fix: Нужно всегда добавлять в init наследуемых объектов keras
    # *args и **kwargs. В данном случае можно вообще не писать 
    # __init__, если не будете добавлять никаких параметров для 
    # своего слоя.
    self.in_shape = kwargs.pop('input_shape')
    super().__init__(*args, **kwargs)

  def call(self, inputs):
    # Совет: чтобы получить целевую размерность, используйте все измерения 
    # тензора, кроме размерности батча, которая не определена на этапе
    # компиляции. Выполните изменение размерности как
    # tf.reshape(inputs, [-1, flattened_dim])
    # где -1 - размерность, которая вычислится динамически.
    # Если нужно преобразовать тип данных от int к float, используйте tf.cast.

    img_shape = inputs.shape[1:]
    assert self.in_shape == img_shape, 'Размерность картинки в батче не совпадает с параметром input_shape'
    
    flattened_dim = np.prod(img_shape)
    _dt = tf.reshape(inputs, [-1, flattened_dim]) 
    out = tf.cast(_dt, tf.float32)/127. - 1.
    return out
    

  def get_config(self):
    # Необязательное задание.
    # Дополните этот метод. Проверьте, работает ли он, при помощи model.save()
    base_config = super().get_config()
    base_config.update({"in_shape": self.in_shape})
    # никаких дополнительных параметров нет, поэтому добавлять нечего
    return base_config

In [47]:
# Можно запустить tensorbaord, если вы подключили TensorBoard callback
# %tensorboard --logdir logs --bind_allupdate

## Код для обучения модели

**FIX** для корректной работы кода загрузки модели: добавлен `input_shape` первому слою. 

**Upd**: поведение может меняться в зависимости от версии 2.x. 

In [48]:
INPUT_SHAPE = (28, 28)

def get_model_with_preparation(n_classes):
  activation = 'tanh'
  units = 256
  model = keras.Sequential([
    DataPreparation(input_shape=INPUT_SHAPE),
    layers.Dense(units, activation=activation),
    layers.Dense(units, activation=activation),
    layers.Dense(n_classes, activation='softmax')])
  return model

In [49]:
model = get_model_with_preparation(10)
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01),
              metrics=['accuracy'],
              loss='sparse_categorical_crossentropy')

Передавайте необработанные данные, т.е. `__x_train`, `__x_test`.

In [50]:
early_stopping = callbacks.EarlyStopping(min_delta=0.001, patience=3, 
                                         restore_best_weights=True)

epochs = 100
batch_size = 128
histrory_prep = model.fit(__x_train, y_train, 
                      validation_data=(__x_test, y_test),
                      epochs=epochs,
                      batch_size=batch_size,
                      callbacks=[early_stopping])

Epoch 1/100
469/469 [==============================] - 1s 2ms/step - loss: 0.7667 - accuracy: 0.7481 - val_loss: 0.5849 - val_accuracy: 0.7939
Epoch 2/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5226 - accuracy: 0.8166 - val_loss: 0.5185 - val_accuracy: 0.8152
Epoch 3/100
469/469 [==============================] - 1s 2ms/step - loss: 0.4738 - accuracy: 0.8332 - val_loss: 0.4888 - val_accuracy: 0.8254
Epoch 4/100
469/469 [==============================] - 1s 2ms/step - loss: 0.4460 - accuracy: 0.8429 - val_loss: 0.4680 - val_accuracy: 0.8317
Epoch 5/100
469/469 [==============================] - 1s 2ms/step - loss: 0.4267 - accuracy: 0.8493 - val_loss: 0.4524 - val_accuracy: 0.8372
Epoch 6/100
469/469 [==============================] - 1s 2ms/step - loss: 0.4121 - accuracy: 0.8547 - val_loss: 0.4406 - val_accuracy: 0.8409
Epoch 7/100
469/469 [==============================] - 1s 2ms/step - loss: 0.4002 - accuracy: 0.8587 - val_loss: 0.4307 - val_accuracy: 0.8443

In [51]:
model.save('m_01')

INFO:tensorflow:Assets written to: m_01/assets


# Задание 2. Слой с гауссовым шумом

Создайте слой, который будет добавлять к входным данным нормально распределенный (с $\mu = 0$ и заданной $\sigma$) шум *и затем*  обрабатывать измененные данные как полносзвязный слой.

Для этого завершите шаблон класса. 

Посоле этого сравните, как обучается модель с различными значениями $\sigma$, к примеру, $0, 0.01, 0.05, 0.1, ...$. Постройте графики в matplotlib или tensorboard. При $\sigma = 0$ слой должен действовать так же, как обычный полносзвязный слой.

Желательно провести несколько экспериментов для каждого значения $\sigma$.

In [162]:
class GaussianDense(layers.Dense):
  def __init__(self, *args, **kwargs):
    # Обратите внимание, что слой наследует Layer -> Dense -> GaussianDense.
    self.std = kwargs.pop('std')
    super().__init__(*args, **kwargs)
    

  def build(self, *args, **kwargs):
    # Достаточно вызова build базового класса.
    super().build(*args, **kwargs)

  def call(self, inputs):
    # Добавить случайный шум, генерируемый методом tf.random.normal.
    # Чтобы определить размерность шума, используйте tf.shape(inputs) - 
    # это поможет избежать сложностей с динамическим размером массива.
    # Вызвать call базового класса можно при помощи super().
    
    _d_gauss = tf.random.normal(tf.shape(inputs), stddev=self.std)
    inputs += _d_gauss
    
    return super().call(inputs)
    

  def get_config(self):
    base_config = super().get_config()
    base_config.update({"std_gauss": self.std})
    return base_config

## Создание и обучение модели

In [163]:
INPUT_SHAPE = (28, 28)

def get_model_with_noise(n_classes, std):
  activation = 'tanh'
  units = 256
  model = keras.Sequential([
    DataPreparation(input_shape=INPUT_SHAPE),
    GaussianDense(units, activation=activation, std=std),
    GaussianDense(units, activation=activation, std=std),
    layers.Dense(n_classes, activation='softmax')])
  return model

In [172]:
def get_callbacks(log_name):
    
    early_stopping = callbacks.EarlyStopping(min_delta=0.001, patience=3, 
                                             restore_best_weights=True)

    tensorboard = tf.keras.callbacks.TensorBoard(log_dir='logs/'+ log_name)
                                              
    
    call_backs = [early_stopping, tensorboard]
    return call_backs

In [173]:
# model = get_model_with_noise(10)
# model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01),
#               metrics=['accuracy'],
#               loss='sparse_categorical_crossentropy')

In [174]:
histrory_gaus_list = list()
std_list = [0, 0.01, 0.05, 0.1, 0.25]
epochs = 100
batch_size = 128

for std in std_list:
    model = get_model_with_noise(10, std=std)
    model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01),
                  metrics=['accuracy'],
                  loss='sparse_categorical_crossentropy')
    
    histrory_gaus = model.fit(__x_train, y_train, 
                          validation_data=(__x_test, y_test),
                          epochs=epochs,
                          batch_size=batch_size,
                          callbacks=get_callbacks(f'std_{std}')
                             )
    
    histrory_gaus_list.append(histrory_gaus)

Epoch 1/100
469/469 [==============================] - 2s 5ms/step - loss: 0.7843 - accuracy: 0.7426 - val_loss: 0.5893 - val_accuracy: 0.7929
Epoch 2/100
469/469 [==============================] - 1s 3ms/step - loss: 0.5240 - accuracy: 0.8185 - val_loss: 0.5186 - val_accuracy: 0.8166
Epoch 3/100
469/469 [==============================] - 1s 3ms/step - loss: 0.4727 - accuracy: 0.8347 - val_loss: 0.4865 - val_accuracy: 0.8284
Epoch 4/100
469/469 [==============================] - 1s 2ms/step - loss: 0.4437 - accuracy: 0.8449 - val_loss: 0.4663 - val_accuracy: 0.8336
Epoch 5/100
 53/469 [==>...........................] - ETA: 0s - loss: 0.4212 - accuracy: 0.8541

KeyboardInterrupt: 

In [158]:
# model.save('gauss_01')

INFO:tensorflow:Assets written to: gauss_01/assets


In [168]:
%load_ext tensorboard
%tensorboard --logdir logs --bind_all

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
